In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/shubo4/CMAL.git

In [ ]:
!pwd
%cd working

In [ ]:
%cd ..
!rm -r CMAL

In [ ]:
%cd CMAL
!ls

In [ ]:
!python train_classify_11_ResNet50.py  --train_path '/kaggle/input/hackathon-dataset/'

In [ ]:
!ls

In [ ]:
%cd outputs

In [ ]:
!ls

In [ ]:

from IPython.display import FileLink 
FileLink(r'model.pth')

In [ ]:
import shutil
shutil.copy('/kaggle/working/CMAL/outputs/results_test.txt','/kaggle/working')

In [ ]:
%cd /kaggle/working

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink 
FileLink(r'results_test.txt')

# DINO finetuning for classification

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
     

In [44]:
def set_seed(no):
    torch.manual_seed(no)
    random.seed(no)
    np.random.seed(no)
    os.environ['PYTHONHASHSEED'] = str()
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(100)

In [45]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/kaggle/input/hackathon-dataset/'
image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir+'/train 2/train'),data_transforms['train']),
                   'test': datasets.ImageFolder(os.path.join(data_dir+'/test 2/test'),data_transforms['test'])}
# dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=32,
#                                              shuffle=True, num_workers=4),
#               'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=32,
#                                              shuffle=True, num_workers=4)}

# dataset_sizes = {'train': len(image_datasets['train']), 'test': len(image_datasets['test'])}
# class_names = image_datasets['train'].classes

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
import torch
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
#dsa = TensorDataset(torch.rand(100, 3), torch.rand(100, 1) )
#dsb = TensorDataset(torch.rand(150, 3), torch.rand(150, 1) )

concat_dataset = ConcatDataset([image_datasets['train'], image_datasets['test']])


In [59]:
loader = DataLoader(concat_dataset,batch_size=32,shuffle=True, num_workers=4)

In [39]:
for i in loader:
    break
    

In [48]:
# load dino model
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:01<00:00, 203MB/s] 


In [60]:
class DinoVisionTransformerClassifier(nn.Module):
    def __init__(self):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.transformer = dinov2_vits14
        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ELU(inplace=True),
            nn.Linear(512, len(class_names)),
        )
    
    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x


In [61]:
model = DinoVisionTransformerClassifier()
model.to(device)

DinoVisionTransformerClassifier(
  (transformer): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, in

In [62]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.00001)

#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 15, eta_min=0)

In [7]:
# !pip install lion-pytorch

In [19]:
# from lion_pytorch import Lion

# optimizer = Lion(model.parameters(), lr=1e-6, weight_decay=1e-2)

In [56]:
for i, data in enumerate(dataloaders["train"], 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        break

In [57]:
outputs.shape

torch.Size([32, 6])

In [16]:
# optimizer = torch.optim.SGD(
#     linear_classifier.parameters(),
#     args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256., # linear scaling rule
#     momentum=0.9,
#     weight_decay=0, # we do not apply weight decay
# )
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs, eta_min=0)

In [ ]:
#CUDA_LAUNCH_BLOCKING=1
model.train()
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for i, data in enumerate(dataloaders["train"], 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        #scheduler.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0

            
print('Finished Training')

[1,    50] loss: 0.093
[1,   100] loss: 0.129
[1,   150] loss: 0.130
[1,   200] loss: 0.103
[2,    50] loss: 0.129
[2,   100] loss: 0.084
[2,   150] loss: 0.107
[2,   200] loss: 0.098


In [21]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in dataloaders["test"]:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.to("cpu") == labels).sum().item()

print(f'Accuracy of the network on the {len(dataloaders["test"])*16} test images: {100 * correct // total} %')

Accuracy of the network on the 64 test images: 100 %


In [20]:
epoch

14

In [21]:
save_dict = {
                "epoch": epoch + 1,
                "state_dict": model.classifier.state_dict(),
                "optimizer": optimizer.state_dict(),
            }
torch.save(save_dict, os.path.join('/kaggle/working', "vitsb_dino_checkpoint.pth.tar"))

In [ ]:
count=0
for i in dataloaders["test"]:
    count+=1

In [ ]:
import torch
torch.save(model,'/kaggle/working/model.pth')

In [22]:
%cd /kaggle/working/

/kaggle/working


In [23]:
!ls

vitsb_dino_checkpoint.pth.tar


In [24]:
from IPython.display import FileLink 
FileLink(r'vitsb_dino_checkpoint.pth.tar')

/kaggle/working/vitsb_dino_checkpoint.pth.tar

## Hyperparameter optimization



In [3]:
!git clone https://github.com/shubo4/Fine-grained_classification.git

Cloning into 'Fine-grained_classification'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [5]:
%cd Fine-grained_classification/

/kaggle/working/Fine-grained_classification


In [6]:
!ls

Finetune_dino.py


In [8]:
!pip install nni

  Obtaining dependency information for nni from https://files.pythonhosted.org/packages/0b/ed/7a061494753317e0943be1685a4690d05e2e610e5b19aa2823547b584950/nni-3.0-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for json-tricks>=3.15.5 from https://files.pythonhosted.org/packages/ae/fd/e3edcf827e7f9c17c5ea1a192841dcfb1dd575a7518c25c5cadd921625b1/json_tricks-3.17.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for typeguard<4.1.3,>=3.0.0 from https://files.pythonhosted.org/packages/bb/bd/dc7da80c95c920ee2b575e64901b5962ca4a1271b5f3cf6c27242aa0aafc/typeguard-4.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.7.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 MB 15.8 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: typing-extensions
  

In [7]:
search_space = {
    'features': {'_type': 'choice', '_value': [128, 256]},
    'lr': {'_type': 'loguniform', '_value': [0.0001, 0.1]},
    "batch_size": {"_type": "choice", "_value": [16,64,128 ]},
}


In [25]:
from nni.experiment import Experiment
experiment = Experiment('local')

experiment.config.trial_command = 'Finetune_dino.py'
experiment.config.trial_code_directory = '.'

experiment.config.search_space = search_space


experiment.config.tuner.name = 'TPE'
experiment.config.tuner.class_args['optimize_mode'] = 'maximize'


# Here we evaluate 10 sets of hyperparameters in total, and concurrently evaluate 2 sets at a time.
experiment.config.max_trial_number = 30
experiment.config.trial_concurrency = 2
# %%

# You can use the web portal to view experiment status: http://localhost:8080.
experiment.run(2020)



[2023-12-03 09:42:53] Creating experiment, Experiment ID: kw9z6jnd
[2023-12-03 09:42:53] Starting web server...
[2023-12-03 09:42:54] Setting up...
[2023-12-03 09:42:54] Web portal URLs: http://127.0.0.1:2020 http://172.19.2.2:2020


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


True

In [30]:
experiment.get_job_metrics(30)

{}

In [21]:
!ls

input  lib  working


In [24]:
experiment.export_data()

[]

In [13]:
import nni
nni.experiment.Experiment.view(experiment,port=8080) 

[2023-12-03 09:28:32] Current config matches the loaded config.
[2023-12-03 09:28:32] Creating experiment, Experiment ID: 83myrl5w
[2023-12-03 09:28:32] Starting web server...
[2023-12-03 09:28:33] Setting up...
[2023-12-03 09:28:33] Web portal URLs: http://127.0.0.1:8080 http://172.19.2.2:8080
[2023-12-03 09:30:10] WARNING: KeyboardInterrupt detected
[2023-12-03 09:30:10] Stopping experiment, please wait...
[2023-12-03 09:30:10] Saving experiment checkpoint...
[2023-12-03 09:30:10] Stopping NNI manager, if any...
[2023-12-03 09:30:10] Experiment stopped.


## Prediction oon test_data

In [22]:

from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd
from PIL import Image
import numpy as np

# Load CSV with Image IDs and Names
csv_path = '/kaggle/input/test-csv/test.csv'
df = pd.read_csv(csv_path)

# Define transformations (adjust according to your model's requirements)
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


# Load PyTorch Model (replace 'YourModelClass' with the actual class of your model)
model.eval()

# Create a custom dataset for inference
class InferenceDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = '/kaggle/input/zip-images/images/'+self.dataframe.iloc[idx]['filename']
        img_id = self.dataframe.iloc[idx]['image_id']

        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, img_id

# Create DataLoader for inference
inference_dataset = InferenceDataset(dataframe=df, transform=transform)
inference_dataloader = DataLoader(inference_dataset, batch_size=8, shuffle=False,num_workers=0)



In [23]:
# Perform Inference and Save Results to CSV
predictions = np.array([])
img_id_arr = np.array([])
with torch.no_grad():
    for images, img_ids in inference_dataloader:

        #images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)

         #outputs = model(images)
         #_, predicted_labels = torch.max(outputs, 1)
        
#         # Append results to the predictions list
        predictions = np.concatenate((predictions, predicted.cpu().numpy()))
        img_id_arr = np.concatenate((img_id_arr,img_ids.cpu().numpy()))


In [25]:
predictions.astype(int).max()

5

In [ ]:
!pwd

In [26]:
import numpy as np
import csv


# Combine the two columns into a 2D numpy array
data = np.column_stack((img_id_arr.astype(int),predictions.astype(int)+1 ))

# Specify the CSV file name
csv_file = "dino_output_vitsb.csv"

# Write the data to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header if needed
    writer.writerow(['image_id','label'])
    
    # Write the data
    writer.writerows(data)

print(f'The data has been successfully written to {csv_file}.')

The data has been successfully written to dino_output_vitsb.csv.


In [28]:
from IPython.display import FileLink 
FileLink(r'dino_output_vitsb.csv')

/kaggle/working/dino_output_vitsb.csv

In [ ]:
num_zeros = np.count_nonzero(predictions == 0)

In [ ]:
import pandas as pd
d = pd.read_csv('dino_output.csv')

In [ ]:
num_zeros = np.count_nonzero(np.array(d['label'])==6)

In [ ]:
num_zeros